<a href="https://colab.research.google.com/github/Srajan1122/Hyperspectral_Image_Classification/blob/srajan/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries

In [0]:
import pandas as pd
import numpy as np
from scipy import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing

# Importing Dataset

In [0]:
loaded_dataset = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/IndianPines/Indian_pines_corrected.mat')
image = loaded_dataset['indian_pines_corrected']

In [0]:
ground_truth = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/IndianPines/Indian_pines_gt.mat')
ground_truth = ground_truth['indian_pines_gt']

## Class Labels

In [0]:
label_values = ["Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]

## Resizing the data

In [0]:
image_with_gt = np.dstack((image, ground_truth))

In [0]:
final_output = image_with_gt.reshape(ground_truth.size, image.shape[2]+1)

## Data Visualization in pandas

In [0]:
data = pd.DataFrame(final_output)

In [43]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
0,3172,4142,4506,4279,4782,5048,5213,5106,5053,4750,4816,4769,4610,4805,4828,4861,4767,4624,4549,4463,4462,4446,4445,4336,4381,4319,4207,4305,4311,3991,4168,3942,4061,4362,4318,4252,4869,5284,5055,3591,...,1396,1381,1396,1381,1353,1346,1341,1332,1324,1310,1318,1330,1310,1292,1280,1275,1266,1264,1233,1241,1232,1215,1215,1187,1168,1171,1150,1134,1123,1135,1094,1090,1112,1090,1062,1069,1057,1020,1020,3
1,2580,4266,4502,4426,4853,5249,5352,5353,5347,5065,5141,5100,4994,5172,5290,5289,5217,5053,5033,4939,4931,4941,4902,4824,4859,4805,4698,4794,4806,4452,4628,4433,4643,4967,4853,4760,5449,5768,5684,3987,...,1421,1415,1428,1415,1379,1370,1360,1353,1352,1336,1346,1351,1330,1315,1305,1292,1282,1286,1259,1259,1250,1229,1232,1195,1177,1184,1153,1137,1138,1137,1108,1104,1117,1091,1079,1085,1064,1029,1020,3
2,3687,4266,4421,4498,5019,5293,5438,5427,5383,5132,5227,5172,5097,5313,5411,5412,5341,5191,5140,5069,5110,5119,5046,4981,5023,4987,4862,4965,4992,4595,4756,4529,4801,5077,4983,4868,5515,5972,5913,4027,...,1446,1440,1443,1425,1390,1379,1376,1363,1355,1347,1361,1356,1341,1330,1321,1304,1290,1289,1263,1269,1261,1245,1241,1214,1185,1188,1156,1147,1149,1144,1111,1114,1114,1100,1065,1092,1061,1030,1016,3
3,2749,4258,4603,4493,4958,5234,5417,5355,5349,5096,5147,5078,5040,5237,5321,5344,5255,5121,5035,4956,4994,4980,4905,4857,4900,4831,4720,4848,4847,4484,4613,4402,4674,4966,4848,4776,5473,5894,5789,4086,...,1432,1427,1426,1416,1386,1374,1375,1359,1343,1343,1354,1351,1333,1329,1313,1296,1280,1281,1251,1255,1253,1238,1223,1207,1188,1188,1154,1143,1144,1146,1122,1108,1109,1109,1071,1088,1060,1030,1006,3
4,2746,4018,4675,4417,4886,5117,5215,5096,5098,4834,4853,4857,4734,4879,4976,4958,4885,4754,4647,4532,4586,4591,4492,4453,4497,4398,4297,4408,4401,4102,4227,4075,4264,4529,4490,4438,5001,5378,5321,3779,...,1401,1397,1395,1390,1368,1349,1354,1340,1330,1324,1336,1332,1320,1307,1287,1283,1267,1265,1239,1240,1239,1229,1212,1202,1178,1178,1143,1135,1138,1135,1110,1107,1112,1094,1072,1087,1052,1034,1019,3


## Droping the rows if ground truth value is zero

In [44]:
print("Percentage of column which will be droped",(data.size - data[data[200] == 0].size)/data.size,"%")

Percentage of column which will be droped 0.48746730083234247 %


In [0]:
data = data[data.iloc[:, -1] != 0]

# Spliting the data into feature and target

In [0]:
X = data.iloc[:, :-1]
y = data.iloc[:,-1]

# Feature Selection

In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

X = SelectKBest(f_classif, k=150).fit_transform(X, y)

# OneHotEncoding in target column

In [0]:
onehotencoder = OneHotEncoder() 
y = onehotencoder.fit_transform(np.array(y).reshape(-1,1)).toarray() 

# Standardizing the feature columna

In [0]:
X = preprocessing.scale(X)

# Spliting the data into training and testing set 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)#0.25 0.15

In [0]:
from keras import backend as K
feature = 150

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, feature)
    X_test = X_test.reshape(X_test.shape[0], 1, feature)
    input_shape = (1, feature)
else:
    X_train = X_train.reshape(X_train.shape[0], feature, 1)
    X_test = X_test.reshape(X_test.shape[0], feature, 1)
    input_shape = (feature, 1)

In [58]:
X_train.shape

(8711, 150)

# Building CNN Model


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D

In [61]:
model = Sequential()
model.add(Conv1D(8, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(16, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(16, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 148, 8)            32        
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 74, 8)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 72, 16)            400       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 36, 16)            0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 36, 16)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)              

In [66]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          epochs=200,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 8711 samples, validate on 1538 samples
Epoch 1/200
8711/8711 [==============================] - 1s 132us/step - loss: 0.8571 - accuracy: 0.6843 - val_loss: 0.6895 - val_accuracy: 0.7666
Epoch 2/200
8711/8711 [==============================] - 1s 114us/step - loss: 0.8499 - accuracy: 0.6835 - val_loss: 0.6469 - val_accuracy: 0.7763
Epoch 3/200
8711/8711 [==============================] - 1s 110us/step - loss: 0.8314 - accuracy: 0.6958 - val_loss: 0.6442 - val_accuracy: 0.7744
Epoch 4/200
8711/8711 [==============================] - 1s 111us/step - loss: 0.8378 - accuracy: 0.6894 - val_loss: 0.6982 - val_accuracy: 0.7432
Epoch 5/200
8711/8711 [==============================] - 1s 111us/step - loss: 0.8260 - accuracy: 0.6951 - val_loss: 0.6527 - val_accuracy: 0.7633
Epoch 6/200
8711/8711 [==============================] - 1s 109us/step - loss: 0.8446 - accuracy: 0.6872 - val_loss: 0.6429 - val_accuracy: 0.7653
Epoch 7/200
8711/8711 [==============================] - 1s 111us/step

In [67]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1538/1538 [==============================] - 0s 56us/step
Test loss: 0.46010912890862116
Test accuracy: 0.8530558943748474
